# Flexible Conditional Generation <a href="https://colab.research.google.com/github/mostly-ai/mostlyai/blob/main/docs/tutorials/conditional-generation/conditional-generation.ipynb" target="_blank"><img src="https://img.shields.io/badge/Open%20in-Colab-blue?logo=google-colab" alt="Run on Colab"></a>

In this tutorial, we show how to generate samples that are conditioned on specific values for a set of attributes. By that, we effectively create partially synthetic data, where synthetic attributes are randomly sampled given the context of a handful of pre-determined fixed attributes. Note, that the synthetic data is still statistically representative, but within the given context. The privacy of the overall dataset is then largely dependend on the privacy of the provided fixed attributes.

We will demonstrate conditional generation for two use cases:
1. First, we generate synthetic data for the UCI Adult Income, but will probe the model for a specific combination of attributes. We will then study the remaining attributes and perform ad-hoc What-If analysis.
1. Secondly, we create partially synthetic data for AirBnB listings in Manhattan, where the locations will then be actual locations, yet all other attributes are synthetic.

To perform either scenario, we will create a Seed table that contains all columns, that we want to hold fixed. Once, a Generator has been created, we can then proceed to provide the seed context with the fixed attributes to conditionally create a Synthetic Dataset.

Note, the same kind of conditional generation can also be performed for two-table setups. Once a two-table Generator is trained, one can simply provide a Seed context for the subject table. The non-fixed columns of the subject table and the entire linked table will then be conditionally generated.

## Use Case 1 - What-If Analysis with UCI Adult Income

For this use case, we will be using again the UCI Adult Income [[1](#refs)] dataset.

### Create a Generator with MOSTLY AI

The code below will create a Generator using the MOSTLY AI Synthetic Data SDK.

In [ ]:
# Install SDK in CLIENT mode
!pip install -U mostlyai
# Or install in LOCAL mode
!pip install -U 'mostlyai[local]'  
# Note: Restart kernel session after installation!

In [ ]:
import pandas as pd

# fetch original data
df = pd.read_csv("https://github.com/mostly-ai/public-demo-data/raw/dev/census/census.csv.gz")
df

In [ ]:
from mostlyai.sdk import MostlyAI

# initialize SDK
mostly = MostlyAI()

In [ ]:
# train a generator on the original training data
g = mostly.train(
    config={
        "name": "Conditional Generation Tutorial Census",
        "tables": [{"name": "data", "data": df}],
    }
)

### Conditionally Probe the Generator

Let's probe the generator by providing a seed dataframe, with specific values for the fixed attributes. In this case, we are interested in studying the common characteristics of 50y male executive manager, with a high-school degree, born in the US.

In [ ]:
record = {
    "age": 50,
    "sex": "Male",
    "occupation": "Exec-managerial",
    "education": "HS-grad",
    "native_country": "United-States",
}
seed = pd.DataFrame([record] * 10_000)
seed

In [ ]:
# probe the generator for synthetic data with a seed
syn_us = mostly.probe(g, seed=seed)
syn_us.head()

In [ ]:
print("50y male Executive Manager, with high-school diploma, born in the United States")
print(f"-> Probability of Earning High Income: {(syn_us.income == '>50K').mean():.1%}")

### What if that same Person is from Mexico? 

In [ ]:
seed_mx = seed.copy()
seed_mx["native_country"] = "Mexico"
syn_mx = mostly.probe(g, seed=seed_mx)

In [ ]:
print("Other things being equal, what if person was born in Mexico?")
print(f"-> Probability of Earning High Income: {(syn_mx.income == '>50K').mean():.1%}")

### What if that same Person is younger / older?

In [ ]:
seed_age_us = seed.copy()
seed_age_mx = seed_mx.copy()
inc_age_us = {}
inc_age_mx = {}
for age in list(range(20, 70, 5)):
    seed_age_us["age"] = age
    seed_age_mx["age"] = age
    syn_age_us = mostly.probe(g, seed=seed_age_us)
    syn_age_mx = mostly.probe(g, seed=seed_age_mx)
    inc_age_us[age] = (syn_age_us.income == ">50K").mean().item()
    inc_age_mx[age] = (syn_age_mx.income == ">50K").mean().item()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(inc_age_us.keys(), inc_age_us.values(), linestyle="-", color="b")
plt.plot(inc_age_mx.keys(), inc_age_mx.values(), linestyle="-", color="g")
plt.title("Earning High Income - Male Executive Manager from US with HS degree")
plt.xlabel("Age")
plt.ylabel("Probability")
plt.tight_layout()
plt.show()

## Use Case 2 - Partially Synthetic Geo Data

For this use case, we will be using 2019 AirBnB listings [2] for Manhattan. The dataset consists of 48,895 records, and 10 mixed-type columns, with two of those representing the latitude and longitude of the listing. We will use this dataset to create synthetic attributes for all the actual locations, that were contained in the original.

### Pre-Processing

We will need to concatenate `latitude` and `longitude` together into a single column, as this is the format expected by MOSTLY AI, in order to improve its representation of geo information.

In this example we will not artifically create Seed data, but will use the concatenated `LAT_LONG` variable and the `neighbourhood` variable from the original data as a Seed dataframe.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# fetch original data
df_orig = pd.read_csv("https://github.com/mostly-ai/public-demo-data/raw/dev/airbnb/airbnb.csv.gz")
df_orig

In [ ]:
df = df_orig.copy()

# concatenate latitude and longitude to "LAT, LONG" format
df["LAT_LONG"] = df["latitude"].astype(str) + ", " + df["longitude"].astype(str)
df = df.drop(columns=["latitude", "longitude"])

# define list of columns, on which we want to condition on
seed_cols = ["neighbourhood", "LAT_LONG"]

# create dataframe that will be used as seed
df_seed = df[seed_cols]
display(df_seed.head())

### Create Generator with MOSTLY AI

The code below will create a Generator using the MOSTLY AI Synthetic Data SDK. We use pre-processed AirBnB data and need to configure column `LAT_LONG` as encoding type `Latitude, Longitude`. In order to not wait too long for the Generator to be ready we are limiting the max. training time to 2 minutes which will already provide us with sufficient quality.

In [ ]:
# Train a generator on the pre-processed AirBnB data
config = {
    "name": "Conditional Generation Tutorial AirBnB",
    "tables": [
        {
            "name": "AirBnB",
            "data": df,
            "columns": [
                {"name": "neighbourhood_group", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "neighbourhood", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "room_type", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "price", "model_encoding_type": "TABULAR_NUMERIC_AUTO"},
                {"name": "minimum_nights", "model_encoding_type": "TABULAR_NUMERIC_AUTO"},
                {"name": "number_of_reviews", "model_encoding_type": "TABULAR_NUMERIC_AUTO"},
                {"name": "last_review", "model_encoding_type": "TABULAR_DATETIME"},
                {"name": "reviews_per_month", "model_encoding_type": "TABULAR_NUMERIC_AUTO"},
                {"name": "availability_365", "model_encoding_type": "TABULAR_NUMERIC_AUTO"},
                {"name": "LAT_LONG", "model_encoding_type": "TABULAR_LAT_LONG"},
            ],
            "tabular_model_configuration": {
                "max_training_time": 2,
            },
        }
    ],
}

g_airbnb = mostly.train(config=config)

### Generate Synthetic Dataset with MOSTLY AI

We can now generate a Synthetic Dataset with the Seed that we already previously created.

In [ ]:
# generate a synthetic dataset with a seed
syn_partial = mostly.probe(generator=g_airbnb, seed=df_seed)
print(f"Created synthetic data with {syn_partial.shape[0]:,} records and {syn_partial.shape[1]:,} attributes")

### Explore Synthetic Data

Let's compare the price distribution across Manhatten. Note again, that while the locations in the partially synthetic data are actual locations, all other attributes, incl. the price per night, are randomly sampled by the generative model. Still, these prices are again statistically representative given the context, i.e. the location within Manhattan.

In [ ]:
%%capture --no-display


def plot_manhatten(df, title):
    ax = df_orig.plot.scatter(
        x="longitude",
        y="latitude",
        s=0.1,
        alpha=1,
        color=np.log(df.price.clip(lower=50, upper=2_000)),
        cmap=plt.colormaps["YlOrRd"],
    )
    ax.set_aspect(1.3)
    ax.set_title(title)
    plt.show()


plot_manhatten(df_orig, "Original Data")
plot_manhatten(syn_partial, "Partially Synthetic Data")

Note, that you can also create a fully synthetic data, and will also yield statistically representative locations with their attributes. However, as these locations do not necessarily exist (e.g. they might end up in the Hudson River), the demonstrated approach allows you to combine the best of both worlds. 

## Conclusion

In this tutorial we walked throught the process of conditional generation to yield partially synthetic data. This allows you to probe the generative model with a specific context, whether that is hypothetical (use case 1) or real (use case 2), and gain corresponding insights for specific scenarios.

## Further exercises

In addition to walking through the above instructions, we suggest..
* to use a different set of fixed columns for the US Census dataset
* to generate a very large number of records for a fixed value set, e.g. create 1 million records of 48 year old female Professors
* to perform a fully synthetic dataset of the AirBnB Manhattan dataset

## References<a class="anchor" name="refs"></a>

1. https://archive.ics.uci.edu/ml/datasets/adult
1. http://insideairbnb.com/get-the-data
